In [1]:
# 기본 
import pandas as pd
import numpy as np
import os, glob, re

# 그래프 
import plotly.express as px
import matplotlib.pyplot as plt   # 맷폴렛- 그래프
%matplotlib inline

# ignore warnings
import warnings    # 워닝 방지
warnings.filterwarnings('ignore')

####### matplotlib - 한글 폰트 깨짐 방지 코드 #######
import platform

if platform.system() == 'Darwin':    # 맥
    font_name = 'AppleGothic'
elif platform.system() == 'Linux':   # 리눅스
    font_name = 'NanumGothic'
elif platform.system() == 'Windows': # 윈도우
    font_name = 'Malgun Gothic'
else:
    print('알수없는 시스템. 미적용')

plt.rc('font', family=font_name)

encoding = ['euc-kr','utf-8', 'CP949', 'utf-8-sig'] 

In [2]:
os.getcwd() # 현재 디렉토리위치

'C:\\studyNote\\projectformDK'

In [3]:
# 변수 
path = 'C:\\studyNote\\projectformDK\\data\\'
name = 'data2.csv'
data_path = glob.glob(path+name)

In [4]:
glob.glob(path+'*')

['C:\\studyNote\\projectformDK\\data\\data.csv',
 'C:\\studyNote\\projectformDK\\data\\data2.csv',
 'C:\\studyNote\\projectformDK\\data\\기상개황_20200922232758.xlsx',
 'C:\\studyNote\\projectformDK\\data\\등록인구추이_20200922232814.xlsx',
 'C:\\studyNote\\projectformDK\\data\\자연재해.xls',
 'C:\\studyNote\\projectformDK\\data\\자연재해복구비.xls',
 'C:\\studyNote\\projectformDK\\data\\지방세_부담_20200922232824.xlsx',
 'C:\\studyNote\\projectformDK\\data\\풍수해_발생_20200922232742.xlsx']

In [6]:
data = pd.read_csv(data_path[0], encoding='cp949')
data.info()
data = data.iloc[:,1:]
data.tail(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      24 non-null     int64  
 1   pay_main  24 non-null     int64  
 2   die       24 non-null     int64  
 3   win_mean  24 non-null     float64
 4   win_max   24 non-null     float64
 5   win_ins   24 non-null     float64
dtypes: float64(3), int64(3)
memory usage: 1.2 KB


,pay_main,die,win_mean,win_max,win_ins
19,94515000,2,3.2,14.6,21.6
20,14141,0,3.1,15.1,25.3
21,66085150,2,3.1,18.6,28.3
22,1803408,0,3.2,15.3,21.9
23,7297329,3,3.2,19.2,33.6


In [ ]:
# data.csv
data_columns_info={
    'pay_main' : '피해액', 
    'die' : '사망자', 
    'victim' : '수재민', 
    'area' : '지역', 
    'pay_dis' :'자연재해피해액(10억 원)',
    'pay_rec' :'총복구액(억)', 
    'tem_mean' : '평균기온',
    'tem_hi' : '최고극값' ,
    'tem_lo' : '최저극값', 
    'rain'   : '강수량', 
    'win_mean':'평균풍속',
    'win_max':'최대풍속', 
    'win_ins':'최대순간풍속',    
}

In [ ]:
x = data.iloc[:, 1:] # 독립변수 
y = data.iloc[:,:1]  #종속 변수 

print(x.shape)
print(y.shape)
# y = target

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

mmsc = MinMaxScaler()
stsc = StandardScaler()

x = mmsc.fit_transform(x)
x = stsc.fit_transform(x)
y = mmsc.fit_transform(y)   # 머신러닝을 위한 정규화
y = stsc.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=5)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
from statsmodels.formula.api import ols

linear_reg = ols('pay_main ~ die + victim + area + pay_dis + pay_rec + tem_mean + tem_hi + tem_lo + rain + win_mean + win_max + win_ins', data=data).fit()
# linear_reg = ols('pay_main ~ die + win_mean + win_max + win_ins', data=data).fit()
linear_reg.summary()

In [ ]:
from sklearn.linear_model import LinearRegression

reg_model = LinearRegression().fit(x_train, y_train)
reg_pred = np.expm1(reg_model.predict(x_test))
print("훈련 세트 정확도: {:.3f}".format(reg_model.score(x_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(reg_model.score(x_test, y_test)))

reg_pred=pd.DataFrame(reg_pred)
plt.plot(y_test, 'ro-', label="real")
plt.plot(reg_pred, 'bs-', label="predict")
plt.legend()
plt.title("Regression")
plt.show()

In [ ]:
!pip install xgboost 

In [ ]:
import xgboost as xgb

xgb = xgb.XGBRegressor(colsample_bytree=0.47,
                       gamma= 0.046,
                      learning_rate=0.05,
                      max_depth=5,
                      min_child_weight=1.8,
                      n_estimators=2000).fit(x_train, y_train)
xgb_pred = np.expm1(xgb.predict(x_test))
print("훈련 세트 정확도: {:.3f}".format(xgb.score(x_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(xgb.score(x_test, y_test)))
print(xgb.feature_importances_)

xgb_pred=pd.DataFrame(xgb_pred)
plt.plot(y_test, 'ro-', label="real")
plt.plot(xgb_pred, 'bs-', label="predict")
plt.legend()
plt.title("XGBoost")
plt.show()

In [ ]:
from sklearn.svm import SVR

svr = SVR(C=1.5,
          cache_size=200,
          coef0=0,
          degree=15,
          kernel='rbf',
          shrinking=True).fit(x_train, y_train)

svr_pred = np.expm1(svr.predict(x_test))
print("훈련 세트 정확도: {:.3f}".format(svr.score(x_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(svr.score(x_test, y_test)))

svr_pred=pd.DataFrame(svr_pred)
plt.plot(y_test, 'ro-', label="real")
plt.plot(svr_pred, 'bs-', label="predict")
plt.legend()
plt.title("SVR")
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=2000).fit(x_train, y_train)

rf_pred = np.expm1(rf.predict(x_test))
print("훈련 세트 정확도: {:.3f}".format(rf.score(x_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(rf.score(x_test, y_test)))
print(rf.feature_importances_)

rf_pred=pd.DataFrame(rf_pred)
plt.plot(y_test, 'ro-', label="real")
plt.plot(rf_pred, 'bs-', label="predict")
plt.legend()
plt.title("RandomForest")
plt.show()